# Explore here

In [38]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

from pickle import dump


from pickle import dump

import streamlit as st

import warnings

In [39]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
df = df[(df['Insulin'] >= 15) & (df['Insulin'] <= 400)]  # mantener valores fisiológicamente válidos

# SkinThickness: eliminar ceros y valores extremos (alta correlación con BMI pero baja con Outcome)
df = df[(df['SkinThickness'] >= 10) & (df['SkinThickness'] <= 60)]

# DiabetesPedigreeFunction: valores > 2 son outliers raros sin mucha correlación con Outcome
df = df[df['DiabetesPedigreeFunction'] <= 2]

In [41]:
X = df.drop("Outcome", axis=1)  
y = df["Outcome"] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_sel = X_train.copy() 
X_test_sel = X_test.copy()   


X_train_sel["Outcome"] = y_train.values 
X_test_sel["Outcome"] = y_test.values   


X_train_sel.to_csv("../data/processed/clean_train.csv", index=False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index=False)

X_train_sel.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
713,0,134,58,20,291,26.4,0.352,21,0
405,2,123,48,32,165,42.1,0.520,26,0
165,6,104,74,18,156,29.9,0.722,41,1
187,1,128,98,41,58,32.0,1.321,33,1
740,11,120,80,37,150,42.3,0.785,48,1


In [42]:
train_data = pd.read_csv("../data/processed/clean_train.csv")
test_data = pd.read_csv("../data/processed/clean_test.csv")

train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,134,58,20,291,26.4,0.352,21,0
1,2,123,48,32,165,42.1,0.520,26,0
2,6,104,74,18,156,29.9,0.722,41,1
3,1,128,98,41,58,32.0,1.321,33,1
4,11,120,80,37,150,42.3,0.785,48,1


In [43]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [44]:
from sklearn import tree
fig = plt.figure(figsize=(15,15))
tree.plot_tree(model, feature_names=list(X_train.columns), class_names = ["0", "1"], filled = True)
plt.show()

In [45]:
y_pred_test = model.predict(X_test)
y_pred_test

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0])

In [46]:
y_pred_train = model.predict(X_train)
y_pred_train

array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0])

In [47]:
dump(model, open("decision_tree_classifier_default_42.sav", "wb"))

In [48]:
accuracy_score(y_test, y_pred_test)

0.6621621621621622

In [51]:
hyperparams = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1,2,4]
 }

grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 10)
grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [None, 'sqrt', 'log2'],
    'class_weight': [None, 'balanced'],
    'random_state': [42]
}
model = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0)

grid_search.fit(X_train, y_train)

print("Mejores parámetros:", grid_search.best_params_)


y_pred_optimizado = grid_search.best_estimator_.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, classification_report

accuracy_optimizado = accuracy_score(y_test, y_pred_optimizado)

f1_micro_optimizado = f1_score(y_test, y_pred_optimizado, average='micro')
f1_macro_optimizado = f1_score(y_test, y_pred_optimizado, average='macro')

report_optimizado = classification_report(y_test, y_pred_optimizado)

print(f"Accuracy del modelo optimizado: {accuracy_optimizado}")
print(f"F1 Score (micro): {f1_micro_optimizado}")
print(f"F1 Score (macro): {f1_macro_optimizado}")
print("\nReporte de clasificación:")
print(report_optimizado)

Mejores parámetros: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'random_state': 42}
Accuracy del modelo optimizado: 0.6621621621621622
F1 Score (micro): 0.6621621621621622
F1 Score (macro): 0.6260359814028704

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.78      0.71      0.74        51
           1       0.46      0.57      0.51        23

    accuracy                           0.66        74
   macro avg       0.62      0.64      0.63        74
weighted avg       0.68      0.66      0.67        74



In [55]:
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [56]:
dump(model, open("../models/tree_classifier_crit-entro_maxdepth-5_minleaf-4_minsplit2_42.sav", "wb"))

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import streamlit as st
import pandas as pd
import numpy as np

# Título de la aplicación
st.title('Mi Primera Aplicación con Streamlit')

# Subtítulo
st.header('Visualización de Datos de Ejemplo')

# Crear un DataFrame de ejemplo
data = pd.DataFrame({
    'Número': np.arange(1, 11),
    'Aleatorio': np.random.randn(10)
})

# Mostrar el DataFrame
st.write("Aquí hay algunos datos de ejemplo:")
st.write(data)

@st.cache(ttl=60*5, max_entries=20)
def load_data():
    data = pd.read_csv('/workspaces/Maria_Miura1_streamlit/data/processed/clean_train.csv')
    return data

data = load_data()

2025-05-14 18:33:17.875 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 18:33:17.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 18:33:17.882 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2025-05-14 18:33:17.883 No runtime found, using MemoryCacheStorageManager
2025-05-14 18:33:17.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 18:33:17.887 Thread 'MainThread': missing ScriptRu